# 001 Preprocess and analyse ECG

### Purpose
Purpose of this notebook is to preprocess the raw ECG data for the 'We Love Reading' study.

- Neurokit2 package with a custom pipeline will be used for preprocessing
- Segmentation is performed afterwards
- QA figures will be exported for every partipant
- The processed signal data will be exported
- HRV metrics are computed and exported


### Input / Output
- Input: `~/data/raw`
- Outputs:
  - QA visualizations: `~/reports/ECG QA`
  - Processed signal data and HRV metrics: `~/data/processed`
  
**Note**:
- Output folders are created if they do not exist
- Make sure the **filenames follows the following format**:
  -  `[condition][id]_W[wavenumber]_mc.txt` for the ECG recordings
  -  `[condition][id]_W[wavenumber]_event.txt` for the corresponding events
  -  ...where 
     -  `[condition]` is a single letter
     -  `[id]` is a number
     -  `[wavenumber]` is a number 
  -  *Example*: B40_W3_mc.txt
  -  Note that it is not case sensitive 


### Imports

In [13]:
# fmt: off
from pathlib import Path
import numpy as np
import importlib
import neurokit2 as nk
import sys
sys.path.append(str(Path().cwd().parent/'src'))
import utils.parameters as params
import utils.common as common
import utils.data_utils as data_utils
import utils.nk_pipeline as nk_pipeline
import app.main as app
importlib.reload(nk)
importlib.reload(nk_pipeline)
importlib.reload(params)
importlib.reload(data_utils)
importlib.reload(common)
import matplotlib.pyplot as plt
plt.ioff()  # Turn off interactive mode
import gc
# fmt:on


# <span style="color:blue">Apply the pipeline</span>

## Set up some Input and Output parameters

In [14]:
WORKING_DIR = Path().cwd()
ROOT_DIR = WORKING_DIR.parent
DATA_DIR = ROOT_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed' # where to save output data 
PROCESSED_DATA_DIR.mkdir(parents=False, exist_ok=True)
QA_REPORTS_DIR = ROOT_DIR / "reports" / 'QA' # where to save QA visualizations
QA_REPORTS_DIR.mkdir(parents=True, exist_ok=True) 

### Example 1: Process a single dyad

**Below an example for how to process and analyse the data from a single dyad.**

Read all ECG datafiles in the Raw data directory

In [15]:
ecg_filepaths = np.sort(list(RAW_DATA_DIR.glob('*mc.txt')))
# [print(f) for f in ecg_filepaths];
# [print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in ecg_filepaths];

Read all Event datafiles in the Raw data directory

In [16]:
event_filepaths = np.sort(list(RAW_DATA_DIR.glob('*event.txt')))
# [print(f) for f in event_filepaths];
# [print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in event_filepaths];

In [17]:
file_index = 0 # whether to process the first, second, third etc. recording

# Preprocess the data, compute HRV metrics and save the output
app.process_dyad(
    ecg_filepath=ecg_filepaths[file_index],
    event_filepath=event_filepaths[file_index],
    parameters=params.base_params,
    data_output_dir=PROCESSED_DATA_DIR,
    figure_output_dir=QA_REPORTS_DIR,
    create_qa_plots=True
)

/Users/lukasspiess/anaconda3/envs/neuroprofile/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


### Example 2: Process all dyads

**Below an example for how to process all dyads, one after another**.  
Much of the preparations needed to process a single dyad is incorporated in the function `app.process_all_dyads()`. In addition, a log file is created in `~/`

In [ ]:
app.process_all_dyads(
    raw_data_dir=RAW_DATA_DIR,
    processed_data_dir=PROCESSED_DATA_DIR,
    reports_dir=QA_REPORTS_DIR,
)

In [ ]:
for index in range(len(ecg_filepaths)):
    try:
        dyad_number, condition, wave = data_utils.extract_subject_id_condition_from_filepath(ecg_filepaths[index])
    except Exception as e:
        print(f"Error processing {ecg_filepaths[index]}: {e}")
        
    print(f"Processing recording {index+1}/{len(ecg_filepaths)}. Dyad number: {dyad_number}. Condition: {condition}. Wave: {wave}")
    nk_pipeline.process_dyad(
        ecg_filepath = ecg_filepaths[index],
        event_filepath = event_filepaths[index],
        parameters=params.base_params,
        data_output_dir=PROCESSED_DATA_DIR,
        figure_output_dir=QA_REPORTS_DIR
    );

    gc.collect()

In [ ]:
for index in range(len(ecg_filepaths)):
    try:
        dyad_number, condition, wave = data_utils.extract_subject_id_condition_from_filepath(ecg_filepaths[index])
        print(f"Processing recording {index+1}/{len(ecg_filepaths)}. Dyad number: {dyad_number}. Condition: {condition}. Wave: {wave}")
        nk_pipeline.process_dyad(
            ecg_filepath = ecg_filepaths[index],
            event_filepath = event_filepaths[index],
            parameters=params.base_params,
            data_output_dir=PROCESSED_DATA_DIR,
            figure_output_dir=QA_REPORTS_DIR
        );
    except Exception as e:
        print(f"Error processing {ecg_filepaths[index]}: {e}")
    gc.collect()


# DEBUG STUFF
|    
V    

In [ ]:
event_filepaths[0]

In [ ]:
ecg_filepaths[0]

# Preprocess first

In [ ]:
# Load and prepare
signal_event_df = data_utils.load_dyad_ecg_events(ecg_filepaths[0], event_filepaths[0])
dyad_parameters = params.configure_segmentation_params(1, params.base_params)
child_params, mother_params = params.configure_ecg_params(1, dyad_parameters)
child_series, mother_series = data_utils.split_in_child_mother_series(signal_event_df)

# Preprocess ECG
child_signals_df = nk_pipeline.ecg_preprocess(child_series, child_params)
mother_signals_df = nk_pipeline.ecg_preprocess(mother_series, mother_params)

# Join the preprocessed signals with the events
child_signal_event_df = child_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")
mother_signal_event_df = mother_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")

In [ ]:
child_signal_event_df.head()

## Then segment and analyze

In [ ]:
# segment the dataframe
child_segments_df_list = data_utils.segment_df(child_signal_event_df, dyad_parameters)
mother_segments_df_list = data_utils.segment_df(mother_signal_event_df, dyad_parameters)

In [ ]:
test_df = child_segments_df_list[0]
test_df[test_df.ECG_R_Peaks == 1].index.diff().dropna()